# 서울 생활이동 데이터 전처리

---

# 0. 라이브러리

In [ ]:
import numpy as np
import pandas as pd

---

# 1. 데이터 불러오기

- 월별/시간대별 유동인구 데이터

In [ ]:
def get_file_path(year, month, time):
    return f'../../data/서울생활이동/생활이동_행정동_202201/생활이동_행정동_{year:04d}.{month:02d}_{time:02d}시.csv'

In [ ]:
year, month = 2022, 1
time_range = range(18, 23)

origins = []
for time in time_range:
    path = get_file_path(year, month, time)
    origin = pd.read_csv(path, encoding='cp949')
    origins.append(origin)

FileNotFoundError: ignored

In [ ]:
origins[0].head()

---

# 2. 합치기

In [ ]:
move = pd.concat(origins, axis=0)
move.shape

---

# 3. 데이터 줄이기

## 1) 평일(월 ~ 금) 추출

In [ ]:
move = move.loc[~move['요일'].isin(['토', '일'])]
move.shape

## 2) 퇴근시간대(18시 ~ 21시)

In [ ]:
# 출발시간 컬럼 생성
move['출발시간'] = move['도착시간'] - move['평균 이동 시간(분)']//60

In [ ]:
# 출발시간을 기준으로 퇴근시간대(18 ~ 21시)만 추출
move = move.loc[move['출발시간'].between(18, 21)]
move.shape

## 3) 서울내 이동

In [ ]:
# 서울->외곽, 외곽->서울 제거
move = move.loc[move['출발 행정동 코드'].between(1100000, 1199999) & move['도착 행정동 코드'].between(1100000, 1199999)]
move.shape

# 4. 전처리

- NaN 없음
- 이동인구(합)에서 `*`은 3명이하의 마스킹 값 -> 평균값(1.5)으로 대체

In [ ]:
move.info()

In [ ]:
# 이동인구(합) '*' -> 1.5

var = '이동인구(합)'
move[var] = move[var].apply(lambda x: '1.5' if x == '*' else x)
move[var] = move[var].astype(float)

# 5. 데이터 저장

In [ ]:
ts, te = time_range.start, time_range.stop - 1
path = f'../../data/서울생활이동/preprocessing/생활이동_행정동_{year:04d}{month:02d}_{ts:02d}-{te:02d}시.csv'
move.to_csv(path, index=False)